In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout

In [ ]:
# Enable GPU memory growth to avoid memory allocation issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:

# Define the discriminator model
def define_discriminator(in_shape=(32,32,3)):
    model = Sequential([
        Conv2D(64, (3,3), padding='same', input_shape=in_shape),
        LeakyReLU(alpha=0.2),
        Conv2D(128, (3,3), strides=(2,2), padding='same'),
        LeakyReLU(alpha=0.2),
        Conv2D(128, (3,3), strides=(2,2), padding='same'),
        LeakyReLU(alpha=0.2),
        Flatten(),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])

        # Compile the discriminator with binary cross-entropy loss
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model


In [ ]:
# Define the generator model
def define_generator(latent_dim):
    model = Sequential([
        Dense(128 * 8 * 8, input_dim=latent_dim),
        LeakyReLU(alpha=0.2),
        Reshape((8, 8, 128)),
        Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
        LeakyReLU(alpha=0.2),
        Conv2D(3, (3,3), activation='tanh', padding='same')
    ])
    return model

In [ ]:
# Define the GAN by combining generator and discriminator
def define_gan(g_model, d_model):
    d_model.trainable = False  # Freeze discriminator during GAN training
    model = Sequential([g_model, d_model])
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return model


In [ ]:
# Load CIFAR-10 dataset and preprocess it
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = trainX.astype('float32')
    X = (X - 127.5) / 127.5  # Normalize images to range [-1,1]
    return X

In [ ]:
# Select real images from the dataset
def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))  # Label real images as 1
    return X, y


In [ ]:
# Generate random latent points for the generator
def generate_latent_points(latent_dim, n_samples):
    return np.random.randn(n_samples, latent_dim)

# Generate fake images using the generator
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = np.zeros((n_samples, 1))  # Label fake images as 0
    return X, y

In [ ]:
# Save generated images at each epoch
def save_plot(examples, epoch, n=7):
    examples = (examples + 1) / 2.0  # Rescale to [0,1] range
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i])
    filename = f'generated_plot_e{epoch+1:03d}.png'
    plt.savefig(filename)
    plt.close()

In [ ]:
# Train the GAN
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=30, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            # Train discriminator on real images
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            # Train discriminator on fake images
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            
            # Train generator (via GAN model)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))  # Labels flipped to 1 to trick the discriminator
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            print(f'>{i+1}, {j+1}/{bat_per_epo}, d1={d_loss1:.3f}, d2={d_loss2:.3f} g={g_loss:.3f}')
        
        # Generate and save images at the end of each epoch
        X_fake, _ = generate_fake_samples(g_model, latent_dim, 49)
        save_plot(X_fake, i)
        print(f'Saved generated images for epoch {i+1}')

In [ ]:
# Run the training process with GPU acceleration
with tf.device('/GPU:0'):
    latent_dim = 100  # Size of the latent space
    d_model = define_discriminator()
    g_model = define_generator(latent_dim)
    gan_model = define_gan(g_model, d_model)
    dataset = load_real_samples()
    train(g_model, d_model, gan_model, dataset, latent_dim)